# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [32]:
# Your code here
salaries.head()
display(salaries.head())
salaries.describe()
display(salaries.describe())

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,25161.000000,8022.000000
mean,34.507604,86786.999790,32.788558
std,9.252077,21041.354602,12.112573
min,10.000000,7200.000000,2.650000
25%,20.000000,76266.000000,21.200000
50%,40.000000,90024.000000,35.600000
75%,40.000000,96060.000000,40.200000
max,40.000000,300000.000000,109.000000


In [33]:
salaries["Department"].value_counts()
salaries["Salary or Hourly"].value_counts()

Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64

Salaries is a DataFrame containing a column Hourly Rate and a column indicating employment type (e.g., Salary or Hourly / Salary Type = "Hourly").

Filter the dataset to include only hourly workers:

In [4]:
hourly_wages = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].dropna()


Get key statistics:

In [5]:
# Sample information

# 1 Sample Size (n):
sample_size = hourly_wages.count()
print("1 Sample Size (n):", sample_size)

# 2 Sample Mean (x̄):
sample_mean = hourly_wages.mean()
print("2 Sample Mean (x̄):", sample_mean)

# 3 Sample Standard Deviation (s)
sample_std_dev = hourly_wages.std()
print("3 Sample Standard Deviation (s)", sample_std_dev)

previous_mean = 180  # what you expected


1 Sample Size (n): 8022
2 Sample Mean (x̄): 32.78855771628024
3 Sample Standard Deviation (s) 12.11257268427681


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [17]:
# Your code here
from scipy.stats import ttest_1samp


# Null Hypothesis (H₀): "The average hourly wage is $30/hr." (Default assumption – you assume this is true unless proven otherwise)
# Alternative Hypothesis (H₁): "The average hourly wage is different from $30/hr."
# Goal: Use your sample data to see if there's strong evidence to reject H₀.
H0 = 30

t_stat, p_value = ttest_1samp(hourly_wages, popmean=H0, alternative='two-sided')

# Output the result
print(f"Test statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.4f} \n")

# For a two-tailed test at 95% confidence:
if p_value < 0.05:
    print("Reject H₀: Average wage ≠ $30/hr")
else:
    print("Fail to reject H₀: No strong evidence")



Test statistic: 20.62
P-value: 0.0000 

Reject H₀: Average wage ≠ $30/hr


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [20]:
# Your code here
confidence_level = 0.95

confidence_interval = t.interval(
    confidence_level,
    df = sample_size,
    loc=sample_mean,
    scale=sample_std_dev / np.sqrt(sample_size) # standard error
)
# 95% confidence interval
ci_lower, ci_upper = confidence_interval

print(f"95% Confidence Interval: ({ci_lower:.2f}, {ci_upper:.2f})")

95% Confidence Interval: (32.52, 33.05)


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [31]:
from statsmodels.stats.proportion import proportions_ztest
# Your code here


# 1 Count Number of hourly workers:
hourly_count = salaries[salaries['Salary or Hourly'] == 'Hourly'].shape[0]

print("1 Number of hourly workers:", hourly_count)

# 2 Total Sample Size (n):
total_count = salaries.shape[0]
print("2 Total Sample Size (n):", total_count)

z_stat, p_value = proportions_ztest(count=hourly_count, nobs = total_count, value=0.25)

print(f"\n p-value = {p_value:.4f}")
if p_value < 0.05:
    print("Conclusion: Proportion is significantly different from 25%")
else:
    print("Conclusion: No significant difference from 25%")


1 Number of hourly workers: 8022
2 Total Sample Size (n): 33183

 p-value = 0.0004
Conclusion: Proportion is significantly different from 25%
